In [ ]:
import datetime

import okx.Account as Account
import okx.Trade as Trade
import okx.MarketData as MarketData
from sqlalchemy.sql.functions import current_timestamp

# API 初始化
apikey = "fe1303e7-a569-448d-a62c-3702046f42fa"
secretkey = "3AF9EBB870593CA49133B7F29BCB53B50"
passphrase = "Wyb199807160518."

flag = "0"  # 实盘:0 , 模拟盘:1

marketDataAPI = MarketData.MarketAPI(flag=flag)
result = marketDataAPI.get_tickers(instType="SPOT")
print(result['data'])

# accountAPI = Account.AccountAPI(apikey, secretkey, passphrase, False, flag)

# result = accountAPI.get_account_balance()
# result

In [58]:
import okx.MarketData as MarketData
from datetime import datetime, timedelta
import pandas as pd
import time

flag = "0"  # 实盘:0 , 模拟盘：1
marketDataAPI = MarketData.MarketAPI(flag=flag)

# 设置参数
instId = "BTC-USDT"
bar = "1D"  # 以天为单位的 K 线数据
limit = 100  # 每次请求最多返回100条数据

# 获取当前时间的 Unix 时间戳
end_time = int(time.time() * 1000)
# 转换为 datetime 对象
current_datetime = datetime.utcfromtimestamp(end_time / 1000)
# 计算一年前的时间
one_year_ago = current_datetime - timedelta(days=365)
# 转换回毫秒级时间戳
start_time = int(one_year_ago.timestamp() * 1000)

# 列表存储所有数据
all_candlesticks = []

while end_time > start_time:
    try:
        # 调用接口获取数据
        result = marketDataAPI.get_history_candlesticks(
            instId=instId,
            after=end_time,  # 请求时间戳之后的数据
            bar=bar,
            limit=str(limit)
        )

        # 检查返回结果
        if 'code' in result and result['code'] != '0':
            print(f"请求失败: {result['code']}, 消息: {result.get('msg', '无错误信息')}")
            break

        # 如果没有数据，退出循环
        if not result or 'data' not in result or not result['data']:
            break

        # 提取并存储 K 线数据
        for item in result['data']:
            candlestick = {
                'ts': item[0],
                'open': item[1],
                'high': item[2],
                'low': item[3],
                'close': item[4],
                'vol': item[5],
                'volCcy': item[6],
                'volCcyQuote': item[7],
                'confirm': item[8]
            }
            all_candlesticks.append(candlestick)

        # 更新 start_time 为返回结果中最后一个数据的时间戳
        end_time = int(result['data'][-1][0])  # 获取最后一个 K 线的时间戳
        # 添加延时，避免频繁请求
        time.sleep(1)  # 每次请求之间等待1秒

    except Exception as e:
        print(f"请求失败: {e}")
        break  # 出现异常则退出循环

# 输出所有获取到的 K 线数据
for candlestick in all_candlesticks:
    print(candlestick)
    
# 将 K 线数据转换为 DataFrame
df = pd.DataFrame(all_candlesticks)
# 转换时间戳为可读日期格式
df['date'] = pd.to_datetime(df['ts'], unit='ms')
# 按时间排序
df.sort_values(by='date', inplace=True)
# 输出至 Excel 文件
df.to_excel('D:/code/quant_code/行情数据/btc_usdt_candlesticks.xlsx', index=False)
print("数据已输出至 btc_usdt_candlesticks.xlsx")


2024-09-26 09:45:44.055 | DEBUG    | okx.utils:get_header_no_sign:39 - header: {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
2024-09-26 09:45:44.056 | DEBUG    | okx.okxclient:_request:42 - domain: https://www.okx.com
2024-09-26 09:45:44.058 | DEBUG    | okx.okxclient:_request:43 - url: /api/v5/market/history-candles?instId=BTC-USDT&after=1727315144055&bar=1D&limit=100
2024-09-26 09:45:44.058 | DEBUG    | okx.okxclient:_request:44 - body:
2024-09-26 09:45:45.378 | DEBUG    | okx.utils:get_header_no_sign:39 - header: {'Content-Type': 'application/json', 'x-simulated-trading': '0'}
2024-09-26 09:45:45.379 | DEBUG    | okx.okxclient:_request:42 - domain: https://www.okx.com
2024-09-26 09:45:45.381 | DEBUG    | okx.okxclient:_request:43 - url: /api/v5/market/history-candles?instId=BTC-USDT&after=1718640000000&bar=1D&limit=100
2024-09-26 09:45:45.382 | DEBUG    | okx.okxclient:_request:44 - body:
2024-09-26 09:45:46.480 | DEBUG    | okx.utils:get_header_no_sign:39 - heade

{'ts': '1727193600000', 'open': '63161', 'high': '64822.2', 'low': '63093.1', 'close': '63499', 'vol': '8036.96094038', 'volCcy': '514081087.171815382', 'volCcyQuote': '514081087.171815382', 'confirm': '1'}
{'ts': '1727107200000', 'open': '63616', 'high': '63950', 'low': '62702.3', 'close': '63160', 'vol': '6765.75462495', 'volCcy': '428267995.64173307', 'volCcyQuote': '428267995.64173307', 'confirm': '1'}
{'ts': '1727020800000', 'open': '62656', 'high': '64750', 'low': '62364.8', 'close': '63616.1', 'vol': '8345.56881174', 'volCcy': '530338826.827120245', 'volCcyQuote': '530338826.827120245', 'confirm': '1'}
{'ts': '1726934400000', 'open': '63170.2', 'high': '63569.1', 'low': '62466.5', 'close': '62656.1', 'vol': '3103.59017277', 'volCcy': '195495869.919978588', 'volCcyQuote': '195495869.919978588', 'confirm': '1'}
{'ts': '1726848000000', 'open': '63159.9', 'high': '63305', 'low': '62338', 'close': '63170.1', 'vol': '3592.07454688', 'volCcy': '226243279.359363969', 'volCcyQuote': '226

C:\Users\华为\AppData\Local\Temp\ipykernel_12660\3045702804.py:76: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['date'] = pd.to_datetime(df['ts'], unit='ms')
